In [2]:
from transformers import AutoTokenizer, BartForConditionalGeneration
from rouge_score import rouge_scorer
import pandas as pd

In [15]:
def summarise(text):
    out_out = ''
    print(len(text.split()))
    for part in range(len(text.split())//1024+1):
        inputs = tokenizer([text[1024*5*part:(part+1)*1024*5]], max_length=1024, return_tensors="pt").to('cuda')
        summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=512).to('cuda')
        out_out = out_out + tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    if len(out_out.split()) <= 350:
        return out_out
    else:
        return summarise(out_out)

In [9]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=40)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [10]:
test_list = pd.read_feather('/home/aikatsnelson/prj1407/dataset/test3.feather')
test_list = test_list['filename']
test_data = test_list.parallel_apply(pd.read_feather)
df = pd.concat(test_data.to_list())

In [11]:
def get_score(vals):
    target, pred = vals['pred'], vals['Annotation']
    scores = scorer.score(target, pred)
    return scores['rouge1'][0], scores['rouge1'][1], scores['rouge1'][2], \
           scores['rouge2'][0], scores['rouge2'][1], scores['rouge2'][2], \
           scores['rouge3'][0], scores['rouge3'][1], scores['rouge3'][2], \
           scores['rougeL'][0], scores['rougeL'][1], scores['rougeL'][2]

In [ ]:
model_names = ['kworts/BARTxiv']
for model_name in model_names:
    print(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name).to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
    df['pred'] = df['Latex'].iloc[:5].apply(summarise)
    df[['rouge1_pr', 'rouge1_rec', 'rouge1_f1', 'rouge2_pr', 'rouge2_rec', 'rouge2_f1', 'rouge3_pr', 'rouge3_rec', 'rouge3_f1', 'rougeL_pr', 'rougeL_rec', 'rougeL_f1']] = df[['pred', 'Annotation']].apply(func=get_score, axis=1, result_type='expand')

kworts/BARTxiv


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


8828
674
8718
506
3228
6605


In [ ]:
df[['rouge1_pr', 'rouge1_rec', 'rouge1_f1', 'rouge2_pr', 'rouge2_rec', 'rouge2_f1', 'rouge3_pr', 'rouge3_rec', 'rouge3_f1', 'rougeL_pr', 'rougeL_rec', 'rougeL_f1']]


In [19]:
scorer.score(df['Annotation'].iloc[0], df['pred'].iloc[0])

{'rouge1': Score(precision=0.165821868211441, recall=0.5654320987654321, fmeasure=0.25643896976483765),
 'rouge2': Score(precision=0.07318840579710145, recall=0.25, fmeasure=0.11322869955156951),
 'rouge3': Score(precision=0.03335750543872371, recall=0.1141439205955335, fmeasure=0.051627384960718295),
 'rougeL': Score(precision=0.07530774800868936, recall=0.25679012345679014, fmeasure=0.11646136618141097)}